In [24]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import re
import requests
import os

In [25]:
navegador = webdriver.Chrome()

In [26]:
navegador.get('https://google.com.br')

In [27]:
barra_pesquisa = navegador.find_element(By.NAME, 'q')

In [28]:
barra_pesquisa.send_keys('Campeonato brasileiro 2025')

In [29]:
barra_pesquisa.send_keys(Keys.ENTER)

In [30]:
link = navegador.find_element(By.PARTIAL_LINK_TEXT, "Tabela e Jogos do Brasileirão Série A 2025")
link.click()

In [31]:
linhasEstatisticas = navegador.find_elements(By.CSS_SELECTOR, ".tabela__pontos tbody tr")
clubes = navegador.find_elements(By.CSS_SELECTOR, ".classificacao__equipes--nome")

In [32]:
campos = [
    "pontos",
    "jogos",
    "vitórias",
    "empates",
    "derrotas",
    "gols_pro",
    "gols_contra",
    "saldo_de_gols",
    "aproveitamento"
]

In [33]:
tabela_classificacao = {}
cabecalho = f"{'Clube':<25} {'P':>3} {'J':>3} {'V':>3} {'E':>3} {'D':>3} {'GP':>4} {'GC':>4} {'SG':>4} {'AP':>5}"

if len(clubes) == len(linhasEstatisticas):
    for clube, estatistica in zip(clubes, linhasEstatisticas):
        valores = estatistica.text.split()
        dados_clube = {campo: valor for campo, valor in zip(campos, valores)}
        tabela_classificacao[clube.text] = dados_clube
else:
    print("Erro: número de clubes e estatísticas não corresponde.")

In [36]:
def mostrar_tabela():
    print(cabecalho)
    print("-" * len(cabecalho))

    for clube, estatisticas in tabela_classificacao.items():
        print(f"{clube:<25} "
            f"{estatisticas['pontos']:>3} "
            f"{estatisticas['jogos']:>3} "
            f"{estatisticas['vitórias']:>3} "
            f"{estatisticas['empates']:>3} "
            f"{estatisticas['derrotas']:>3} "
            f"{estatisticas['gols_pro']:>4} "
            f"{estatisticas['gols_contra']:>4} "
            f"{estatisticas['saldo_de_gols']:>4} "
            f"{estatisticas['aproveitamento']:>4}%")

In [15]:
print("Times disponíveis:\n")
for i, clube in enumerate(tabela_classificacao.keys(), start=1):
    print(f"{i}. {clube}")

escolha = int(input("\nDigite o número do time desejado: ")) - 1

nomes_times = list(tabela_classificacao.keys())
if 0 <= escolha < len(nomes_times):
    time = nomes_times[escolha]
    dados = tabela_classificacao[time]

    texto = (
        f"\n{time} disputou {dados['jogos']} jogos.\n"
        f"Conquistou {dados['vitórias']} vitórias, "
        f"{dados['empates']} empates e {dados['derrotas']} derrotas.\n"
        f"Marcou {dados['gols_pro']} gols e sofreu {dados['gols_contra']}.\n"
        f"Tem um saldo de gols de {dados['saldo_de_gols']}.\n"
        f"Está na {escolha+1} posição com {dados['pontos']} pontos e um aproveitamento de {dados['aproveitamento']}%."
    )
    print(texto)
else:
    print("Número inválido. Por favor, escolha um número da lista.")


Times disponíveis:

1. Flamengo
2. Cruzeiro
3. Bragantino
4. Palmeiras
5. Fluminense
6. Bahia
7. Mirassol
8. Atlético-MG
9. Botafogo
10. Ceará
11. Corinthians
12. Grêmio
13. São Paulo
14. Internacional
15. Vasco
16. Vitória
17. Fortaleza
18. Santos
19. Juventude
20. Sport

Bragantino disputou 11 jogos.
Conquistou 7 vitórias, 2 empates e 2 derrotas.
Marcou 14 gols e sofreu 8.
Tem um saldo de gols de 6.
Está na 3 posição com 23 pontos e um aproveitamento de 69%.


In [ ]:
jogos = navegador.find_elements(By.CSS_SELECTOR, ".lista-jogos__jogo")


In [ ]:
jogos_disponiveis = []


print("Selecione um jogo:\n")

for idx, jogo in enumerate(jogos):
    linhas = jogo.text.split('\n')
    linhas = [linha for linha in linhas if linha.strip() and "FIQUE POR DENTRO" not in linha]

    if len(linhas) >= 3:
        linha0 = linhas[0]
        padrao = re.compile(r"(.*?)(\d{2}/\d{2})([A-Za-zÀ-ÿ]+)?(\d{2}:\d{2})")
        m = padrao.match(linha0)

        if m:
            local = m.group(1).strip()
            data = m.group(2)
            dia = m.group(3) if m.group(3) else ""
            hora = m.group(4)

            if len(linhas) == 3:
                # Jogo futuro (sem placar)
                time_casa = linhas[1]
                time_visitante = linhas[2]
                placar = None
            elif len(linhas) >= 6:
                # Jogo encerrado
                time_casa = linhas[1]
                gols_casa = linhas[2]
                gols_visitante = linhas[3]
                time_visitante = linhas[4]
                placar = f"{gols_casa} x {gols_visitante}"
            else:
                print(f"{idx + 1}. Formato inesperado (linhas insuficientes):", linhas)
                continue

            jogos_disponiveis.append({
                "time_casa": time_casa,
                "time_visitante": time_visitante,
                "local": local,
                "data": data,
                "dia": dia,
                "hora": hora,
                "placar": placar
            })

            texto_extra = f" - Placar: {placar}" if placar else ""
            print(f"{idx + 1}. {time_casa} x {time_visitante} - {data} {hora} ({local}){texto_extra}")
        else:
            print(f"{idx + 1}. Erro ao ler local/data/hora: {linha0}")
    else:
        print(f"{idx + 1}. Formato inesperado:", linhas)

# Leitura da escolha
while True:
    try:
        escolha = int(input("\nDigite o número do jogo desejado: "))
        if 1 <= escolha <= len(jogos_disponiveis):
            break
        else:
            print("Número fora do intervalo.")
    except ValueError:
        print("Entrada inválida. Digite um número.")

# Jogo escolhido
jogo_selecionado = jogos_disponiveis[escolha - 1]

dadosCasa = tabela_classificacao[jogo_selecionado['time_casa']]
dadosVisita = tabela_classificacao[jogo_selecionado['time_visitante']]

texto = (
        f"\n📊 Desempenho Atual do {jogo_selecionado['time_casa']}:\n"
        f"- Jogos disputados: {dadosCasa['jogos']}\n"
        f"- Vitórias: {dadosCasa['vitórias']}, Empates: {dadosCasa['empates']}, Derrotas: {dadosCasa['derrotas']}\n"
        f"- Gols marcados: {dadosCasa['gols_pro']}, Gols sofridos: {dadosCasa['gols_contra']}\n"
        f"- Saldo de gols: {dadosCasa['saldo_de_gols']}\n"
        f"- Pontuação: {dadosCasa['pontos']} pontos\n"
        f"- Aproveitamento: {dadosCasa['aproveitamento']}%\n"
    )

# Prompt para API ou exibição
prompt = (
    f"Considere o jogo entre {jogo_selecionado['time_casa']} e {jogo_selecionado['time_visitante']}.\n"
    f"Local: {jogo_selecionado['local']}\n"
    f"Data: {jogo_selecionado['data']} {jogo_selecionado['dia']}\n"
    f"Hora: {jogo_selecionado['hora']}.\n\n"
    f"\nDesempenho Atual do {jogo_selecionado['time_casa']}:\n"
    f"- Jogos disputados: {dadosCasa['jogos']}\n"
    f"- Vitórias: {dadosCasa['vitórias']}, Empates: {dadosCasa['empates']}, Derrotas: {dadosCasa['derrotas']}\n"
    f"- Gols marcados: {dadosCasa['gols_pro']}, Gols sofridos: {dadosCasa['gols_contra']}\n"
    f"- Saldo de gols: {dadosCasa['saldo_de_gols']}\n"
    f"- Pontuação: {dadosCasa['pontos']} pontos\n"
    f"- Aproveitamento: {dadosCasa['aproveitamento']}%\n"
    f"\nDesempenho Atual do {jogo_selecionado['time_visitante']}:\n"
    f"- Jogos disputados: {dadosVisita['jogos']}\n"
    f"- Vitórias: {dadosVisita['vitórias']}, Empates: {dadosCasa['empates']}, Derrotas: {dadosCasa['derrotas']}\n"
    f"- Gols marcados: {dadosVisita['gols_pro']}, Gols sofridos: {dadosCasa['gols_contra']}\n"
    f"- Saldo de gols: {dadosVisita['saldo_de_gols']}\n"
    f"- Pontuação: {dadosVisita['pontos']} pontos\n"
    f"- Aproveitamento: {dadosVisita['aproveitamento']}%\n"
    "Baseado no desempenho recente dos dois times e o local do jogo, faça uma previsão realista do resultado da partida."
)

print(prompt)


Selecione um jogo:

1. Fluminense x Vasco - 24/05 18:30 (Maracanã) - Placar: 2 x 1
2. São Paulo x Mirassol - 24/05 18:30 (Morumbis) - Placar: 0 x 2
3. Atlético-MG x Corinthians - 24/05 21:00 (Arena MRV) - Placar: 0 x 0
4. Grêmio x Bahia - 25/05 11:00 (Arena do Grêmio) - Placar: 1 x 0
5. Palmeiras x Flamengo - 25/05 16:00 (Allianz Parque) - Placar: 0 x 2
6. Sport x Internacional - 25/05 16:00 (Ilha do Retiro) - Placar: 1 x 1
7. Vitória x Santos - 25/05 18:30 (Barradão) - Placar: 0 x 1
8. Fortaleza x Cruzeiro - 25/05 20:30 (Castelão (CE)) - Placar: 0 x 2
9. Bragantino x Juventude - 26/05 20:00 (Cícero de Souza Marques) - Placar: 1 x 0
10. Botafogo x Ceará - 04/06 20:00 (Nilton Santos (Engenhão))

Prompt para previsão:

Considere o jogo entre Botafogo e Ceará.
Local: Nilton Santos (Engenhão)
Data: 04/06 Quarta
Hora: 20:00.

Baseado no desempenho recente dos dois times e o local do jogo, faça uma previsão realista do resultado da partida.


In [ ]:

API_KEY = "sUA CHAVE"

# Endpoint da API ChatGPT (GPT-4)
url = "https://api.openai.com/v1/chat/completions"

# Função para chamar a API e obter uma previsão
def obter_previsao_jogo():

    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type": "application/json"
    }

    body = {
        "model": "gpt-3.5-turbo",
        "messages": [
            {"role": "system", "content": "Você é um especialista em futebol brasileiro."},
            {"role": "user", "content": prompt}
        ],
        "temperature": 0.7
    }

    response = requests.post(url, headers=headers, json=body)

    if response.status_code == 200:
        resposta = response.json()
        return resposta["choices"][0]["message"]["content"]
    else:
        return f"Erro {response.status_code}: {response.text}"
previsao = obter_previsao_jogo()
print("\n🔮 Previsão da IA para o jogo:\n")
print(previsao)


🔮 Previsão da IA para o jogo:

Erro 429: {
    "error": {
        "message": "You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.",
        "type": "insufficient_quota",
        "param": null,
        "code": "insufficient_quota"
    }
}



In [ ]:
def exibir_menu():
    print("\n==== MENU PRINCIPAL ====")
    print("1 - Mostrar tabela")
    print("2 - Jogos da rodada")
    print("3 - Sair")

def main():
    while True:
        exibir_menu()
        opcao = input("Escolha uma opção: ")

        if opcao == '1':
            mostrar_tabela()
        elif opcao == '2':
            a = int(input("Digite o primeiro número: "))
            b = int(input("Digite o segundo número: "))
            print(f"Soma: {a + b}")
        elif opcao == '3':
            print("Saindo do programa...")
            break
        else:
            print("Opção inválida. Tente novamente.")

if __name__ == "__main__":
    main()


==== MENU PRINCIPAL ====
1 - Mostrar mensagem
2 - Calcular soma
3 - Sair
Clube                       P   J   V   E   D   GP   GC   SG    AP
------------------------------------------------------------------
Flamengo                   24  11   7   3   1   24    4   20   72%
Cruzeiro                   23  11   7   2   2   17    8    9   69%
Bragantino                 23  11   7   2   2   14    8    6   69%
Palmeiras                  22  11   7   1   3   12    8    4   66%
Fluminense                 20  11   6   2   3   15   12    3   60%
Bahia                      18  11   5   3   3   11   11    0   54%
Mirassol                   17  11   4   5   2   17   12    5   51%
Atlético-MG                17  11   4   5   2   11   10    1   51%
Botafogo                   15  10   4   3   3   11    5    6   50%
Ceará                      15  10   4   3   3   11    8    3   50%
Corinthians                15  11   4   3   4   12   14   -2   45%
Grêmio                     15  11   4   3   4   11   14